In [0]:
import csv
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# This is v4: Filter NULLS, fill mean-mode, without transformation, without handling outliers
train=pd.read_csv('gdrive/My Drive/Colab Notebooks/Fraud/Data/v4_filterNulls_fill_mean_mode.csv', compression='gzip')

In [0]:
train.drop('P_emaildomain',axis=1,inplace=True)
train.drop('R_emaildomain',axis=1,inplace=True)
train.drop('DeviceInfo',axis=1,inplace=True)

In [0]:
train.isnull().sum().sum()

0

In [0]:
train['isFraud'].value_counts(dropna=False)

0    569877
1     20663
Name: isFraud, dtype: int64

In [0]:
data_isFraud_1_indexes=list(train[train['isFraud']==1].index)
data_isFraud_0_indexes=list(train[train['isFraud']==0].index)

In [0]:
data_isFraud_1=train.iloc[data_isFraud_1_indexes , : ]
data_isFraud_0=train.iloc[data_isFraud_0_indexes , : ]

In [0]:
target_isFraud_1=data_isFraud_1['isFraud']
print('target_isFraurd_1 shape: ',target_isFraud_1.shape)

features_isFraud_1=data_isFraud_1.drop(['isFraud'],axis=1)
print('features_isFraud_1 shape: ',features_isFraud_1.shape)

target_isFraud_0=data_isFraud_0['isFraud']
print('target_isFraurd_0 shape: ',target_isFraud_0.shape)

features_isFraud_0=data_isFraud_0.drop(['isFraud'],axis=1)
print('features_isFraud_0 shape: ',features_isFraud_0.shape)

target_isFraurd_1 shape:  (20663,)
features_isFraud_1 shape:  (20663, 364)
target_isFraurd_0 shape:  (569877,)
features_isFraud_0 shape:  (569877, 364)


In [0]:
# split data into train and test sets
def split_data(features,target,test_percentage,seed):
  # split data into train and test sets
  seed = 7  # random number in order to get same results each time 
  test_size = 0.3 # Use 30% for test
  X_train, X_test, Y_train, Y_test= train_test_split(features, target, test_size=test_size, random_state=seed)
  print('X_train shape: ' + str(X_train.shape))
  print('Y_train shape: ' + str(Y_train.shape))
  print('X_test shape: ' + str(X_test.shape))
  print('Y_test shape: ' + str(Y_test.shape))
  return  X_train, X_test, Y_train, Y_test


In [0]:
seed = 7  # random number in order to get same results each time 
test_percent = 0.3 # Use 30% for test

X_train_isFraud_1, X_test_isFraud_1, Y_train_isFraud_1, Y_test_isFraud_1 = split_data(features_isFraud_1, target_isFraud_1, test_percent, seed)

X_train shape: (14464, 364)
Y_train shape: (14464,)
X_test shape: (6199, 364)
Y_test shape: (6199,)


In [0]:
seed = 7  # random number in order to get same results each time 
test_size = 0.3 # Use 30% for test

X_train_isFraud_0, X_test_isFraud_0, Y_train_isFraud_0, Y_test_isFraud_0 = split_data(features_isFraud_0, target_isFraud_0, test_percent, seed)

X_train shape: (398913, 364)
Y_train shape: (398913,)
X_test shape: (170964, 364)
Y_test shape: (170964,)


In [0]:
train_features = pd.concat([X_train_isFraud_0, X_train_isFraud_1])
train_target = pd.concat([Y_train_isFraud_0, Y_train_isFraud_1])
test_features = pd.concat([X_test_isFraud_0, X_test_isFraud_1])
test_target = pd.concat([Y_test_isFraud_0, Y_test_isFraud_1])

In [0]:
train=pd.concat([train_target,train_features], axis=1, join='outer')

In [0]:
test=pd.concat([test_target,test_features], axis=1, join='outer')

In [0]:
train

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,_Weekdays,_Hours,_Days,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,device_name,had_id,_Month
539350,0,117.000,W,9973,310.0,150.0,mastercard,224.0,debit,512.0,87.0,119.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,290.0,290.0,89.0,139.0,89.0,0.0,291.0,291.0,T,T,T,M0,F,F,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,0,16,14,yahoo,com,google,com,Windows,1,5
293488,0,107.950,W,2927,531.0,150.0,visa,226.0,debit,469.0,87.0,119.0,4.0,2.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,0.0,2.0,0.0,21.0,3.0,192.0,192.0,10.0,378.0,10.0,378.0,147.0,378.0,T,T,T,M0,F,F,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,3,20,22,google,com,google,com,Windows,1,2
22237,0,50.000,W,12839,321.0,150.0,visa,226.0,debit,264.0,87.0,119.0,4.0,3.0,0.0,0.0,1.0,4.0,0.0,0.0,3.0,0.0,4.0,0.0,29.0,3.0,489.0,475.0,20.0,60.0,20.0,489.0,147.0,489.0,T,T,T,M0,F,T,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,3,17,7,google,com,google,com,Windows,1,12
329358,0,554.000,W,14502,191.0,150.0,visa,226.0,debit,143.0,87.0,3.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,16.0,0.0,1.0,17.0,1.0,1.0,17.0,17.0,T,T,T,M1,F,F,...,554.000000,0.000000,554.000000,0.000000,0.000000,0.000000,0.000000,0.0,24.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,6,19,4,google,com,google,com,Windows,1,3
270018,0,128.272,C,16746,500.0,185.0,mastercard,102.0,credit,299.0,87.0,119.0,1.0,3.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,4.0,4.0,0.0,4.0,0.0,0.0,147.0,4.0,T,T,T,M2,F,F,...,20.965700,20.965700,20.965700,20.965700,128.272202,128.272202,128.272202,0.0,0.0,0.0,-5.0,348684.0,0.0,0.0,100.000000,NotFound,52.0,Found,Found,225.0,266.0,305.0,Found,Found,Chrome,F,F,T,F,desktop,2,19,14,other,com,other,com,Windows,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400289,1,60.420,C,9917,142.0,185.0,visa,138.0,debit,299.0,87.0,119.0,5.0,6.0,0.0,1.0,0.0,1.0,1.0,5.0,0.0,2.0,3.0,3.0,0.0,0.0,0.0,170.0,0.0,0.0,0.0,0.0,147.0,0.0,T,T,T,M2,F,F,...,0.000000,0.000000,0.000000,0.000000,60.419998,60.419998,60.419998,0.0,0.0,0.0,-5.0,55597.0,0.0,-5.0,100.000000,NotFound,52.0,Found,Found,225.0,176.0,595.0,Found,Found,Chrome,F,F,T,F,desktop,1,20,27,google,com,google,com,Windows,1,3
32333,1,33.697,C,4461,375.0,185.0,mastercard,224.0,debit,511.0,60.0,119.0,2.0,2.0,0.0,2.0,0.0,2.0,2.0,1.0,0.0,2.0,2.0,2.0,0.0,0.0,1.0,170.0,1.0,0.0,42.0,0.0,147.0,0.0,T,T,T,M1,F,F,...,0.000000,0.000000,0.000000,0.000000,38.478001,38.478001,38.478001,0.0,0.0,0.0,-5.0,118970.0,0.0,0.0,100.000000,NotFound,52.0,Found,Found,225.0,153.0,563.0,Found,Found,Chrome,F,F,T,T,desktop,6,3,10,google,com,google,com,Windows,1,12
193931,1,100.000,H,8406,264.0,150.0,visa,226.0,debit,337.0,87.0,119.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,170.0,28.0,140.0,42.0,124.0,147.0,164.0,T,T,T,M0,F,F,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-5.0

In [0]:
test

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,_Weekdays,_Hours,_Days,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,device_name,had_id,_Month
272748,0,189.950,W,13534,105.0,150.0,visa,226.0,debit,220.0,87.0,119.0,1.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,147.0,2.0,T,T,T,M0,F,F,...,92.949997,92.949997,92.949997,92.949997,0.000000,92.949997,92.949997,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,3,16,15,yahoo,com,google,com,Windows,1,2
501349,0,27.950,W,4663,490.0,150.0,visa,166.0,debit,251.0,87.0,70.0,10.0,20.0,0.0,0.0,11.0,11.0,0.0,0.0,12.0,0.0,12.0,0.0,53.0,10.0,6.0,6.0,6.0,0.0,42.0,6.0,6.0,6.0,T,T,F,M0,F,F,...,141.949997,141.949997,141.949997,141.949997,0.000000,0.000000,0.000000,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,2,0,2,google,com,google,com,Windows,1,5
55141,0,407.500,W,8701,111.0,150.0,visa,226.0,debit,310.0,87.0,10.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,170.0,1.0,1.0,1.0,1.0,1.0,1.0,T,T,T,M1,F,F,...,0.000000,0.000000,0.000000,0.000000,1369.000000,1369.000000,1369.000000,407.5,407.5,407.5,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,4,20,15,aol,com,google,com,Windows,1,12
253331,0,29.000,W,7069,111.0,150.0,mastercard,202.0,debit,325.0,87.0,7.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,170.0,28.0,0.0,42.0,0.0,0.0,0.0,T,T,T,M1,F,F,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,3,23,8,google,com,google,com,Windows,1,2
133711,0,54.500,W,7919,194.0,150.0,mastercard,202.0,debit,181.0,87.0,22.0,1.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,1.0,263.0,263.0,242.0,0.0,42.0,0.0,0.0,0.0,T,T,T,M0,T,F,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,6,21,31,google,com,google,com,Windows,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286801,1,59.000,W,2616,327.0,150.0,discover,102.0,credit,204.0,87.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,1.0,9.0,1.0,72.0,72.0,5.0,73.0,5.0,73.0,68.0,73.0,T,F,F,M0,T,F,...,59.000000,0.000000,0.000000,0.000000,0.000000,49.000000,49.000000,0.0,0.0,0.0,-10.0,174717.0,2.0,-7.0,99.745325,NotFound,52.0,Found,Found,166.0,266.0,507.0,Found,Found,Chrome,T,F,T,F,desktop,1,16,20,google,com,google,com,Windows,1,2
544156,1,24.715,C,15885,545.0,185.0,visa,138.0,debit,299.0,87.0,119.0,3.0,22.0,0.0,2.0,0.0,2.0,2.0,1.0,0.0,1.0,2.0,2.0,1.0,1.0,3.0,3.0,0.0,3.0,0.0,0.0,147.0,3.0,T,T,T,M2,F,F,...,25.452700,25.452700,25.452700,25.452700,97.753197,669.395386,669.395386,0.0,0.0,0.0,-25.0,164198.0,0.0,0.0,100.000000,NotFound,64.0,Found,Found,225.0,266.0,325.0,Found,Found,Chrome,F,F,T,F,mobile,2,1,16,google,com,google,com,Huawei,1,5
236190,1,67.067,C,5943,296.0,185.0,visa,195.0,debit,299.0,87.0,119.0,194.0,261.0,0.0,52.0,0.0,52.0,52.0,132.0,0.0,49.0,145.0,145.0,14.0,11.0,49.0,49.0,28.0,1.0,0.0,0.0,147.0,0.0,T,T,T,M2,F,F,...,0.000000,0.000000,0.000000,0.000000,0.000000,15.857600,0.000000,0.0,0.0,0.0,-25.0,91

In [0]:
train.to_csv('gdrive/My Drive/Colab Notebooks/Fraud/Data/v4_train_data',index=False,compression='gzip')

In [0]:
test.to_csv('gdrive/My Drive/Colab Notebooks/Fraud/Data/v4_test_data.csv',index=False,compression='gzip')